In [1]:
import os
import pandas as pd
from dotenv import load_dotenv
from utils.coin import CoinGeckoAPI  
from utils.manager import BitcoinDatabase  

load_dotenv()

db_name = os.getenv("BTC_DB")
table_btc_price_history = os.getenv("BTC_PRICE_TABLE")
table_rolling_price_history = os.getenv("ROLLING_PRICE_TABLE")



c:\users\toglia\appdata\local\programs\python\python38\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


##  Load the data from the db 

In [21]:
btc_db = BitcoinDatabase(db_name,table_btc_price_history)
btc_history_data = btc_db.select_data(table_btc_price_history)



## Find peak values calculating std

If a value is greater or less than two times the standard deviation, it can be considered an anomaly 

In [22]:
df = btc_history_data.copy()

In [23]:
df.describe()

,price
count,2161.000000
mean,41286.376671
std,3085.657626
min,33505.713900
25%,38799.134300
50%,41702.562600
75%,43383.780100
max,48033.245200


In [24]:
desviacion_estandar = df['price'].std()

# Encontrar los valores que están por encima o por debajo de 2 desviaciones estándar
valores_anomalos = df[(df['price'] > (df['price'].mean() + 2 * desviacion_estandar)) | 
                      (df['price'] < (df['price'].mean() - 2 * desviacion_estandar))]

# Etiquetar los valores anómalos en el DataFrame original
df['anomalo'] = False
df.loc[valores_anomalos.index, 'anomalo'] = True




## Anomaly values

In [25]:
df[df['anomalo']==True]

,date,price,anomalo
14,01-01-2022 14:06:31,48033.2452,True
15,01-01-2022 15:03:26,47648.6577,True
17,01-01-2022 17:06:07,47490.0659,True
19,01-01-2022 19:04:22,47506.7727,True
20,01-01-2022 20:01:18,47816.0777,True
21,01-01-2022 21:04:15,47692.3364,True
23,01-01-2022 23:00:06,47482.1361,True
37,02-01-2022 13:02:39,47861.4554,True
56,03-01-2022 08:00:33,47503.7562,True
83,04-01-2022 11:01:44,47507.6937,True


In [29]:
# Processing date to obtain more info

df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y %H:%M:%S')
df['day'] = df['date'].apply(lambda x: x.day)
df['month'] = df['date'].apply(lambda x: x.month)
df['week'] = df['date'].apply(lambda x: x.isocalendar()[1])

In [30]:
# Agrupar los datos por día, mes y semana y calcular la variación del precio de Bitcoin
meses_variacion = df.groupby('month')['price'].std().sort_values(ascending=False)
semanas_variacion = df.groupby('week')['price'].std().sort_values(ascending=False)



print("Semanas con mayor variación en el precio de Bitcoin:")
print(semanas_variacion.to_frame()[0:3] )

print("Mes con mayor variación en el precio de Bitcoin:")
print(meses_variacion.to_frame())

Semanas con mayor variación en el precio de Bitcoin:
            price
week             
3     3044.282615
9     2216.861163
1     2179.998169
Mes con mayor variación en el precio de Bitcoin:
             price
month             
1      3623.356522
3      2815.652961
2      2535.325682


## Calculating the IQR 

In [31]:
# Calcular el primer y tercer cuartil
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)

# Calcular el rango intercuartil
iqr = q3 - q1

# Imprimir el rango intercuartil
print("El rango intercuartil es:", iqr)

El rango intercuartil es: 4584.645800000006


## According the IQR obtained the 50% of the values  have a spread of  4584.6458$